<a href="https://colab.research.google.com/github/amnasyed1/datasci_6_anova/blob/main/datasci_6_anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ucimlrepo

In [39]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 1. Data Preparation


In [ ]:
# fetch dataset
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)

# data (as pandas dataframes)
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets

/usr/local/lib/python3.10/dist-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


# Variables of Interest

**Research Question:** How do the race of the patient (race) and their A1C test results (A1Cresult) jointly influence the duration of their hospital stay (time_in_hospital)?

**Independent Variable 1 (iv1) - Race**

(Race - race): Caucasian, Asian, African American, Hispanic, Other

**Independent Variable 2 (iv2)- A1C Results**

(A1C test result - a1c):	>8%, 7%, Normal, None (not measure)

**Dependent Variable:** Duration hospital stay (time_in_hospital)

In [4]:
df = pd.DataFrame(X)
df

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,No
1,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
2,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,...,No,No,No,No,No,No,No,No,No,Yes
3,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,...,No,No,Steady,No,No,No,No,No,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101762,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,MC,NaN,...,No,No,Steady,No,No,No,No,No,No,Yes
101763,Caucasian,Male,[70-80),NaN,1,1,7,1,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101764,Caucasian,Female,[80-90),NaN,2,3,7,10,MC,Surgery-General,...,No,No,Up,No,No,No,No,No,Ch,Yes


In [6]:
df.columns

Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed'],
      dtype='object')

In [11]:
# Create a subset of the data that includes only the variables of interest

variables_of_interest = ['race', 'A1Cresult', 'time_in_hospital']
diabetes = df[variables_of_interest]

In [20]:
# Ensuring the data types are appropriate
data_types = diabetes.dtypes
data_types

race                object
A1Cresult           object
time_in_hospital     int64
dtype: object

In [27]:

missing_values = diabetes.isnull().sum()
missing_values

race                0
A1Cresult           0
time_in_hospital    0
dtype: int64

In [18]:
diabetes.shape

(101766, 3)

In [28]:
df.to_csv('diabeteswk6.csv')


When I downloaded the csv file, and viewed it from my desktop, under the 'race' column there were missing values; therefore, I had to re-evaluate the missing values results. The 'A1Cresult' and 'time_in_hospital' columns did not have any missing values, and to double check there were no missing values in the downloaded csv file either.

In [32]:
df_diabetes = df.dropna(subset=['race'])
df_diabetes

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,No
1,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
2,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,...,No,No,No,No,No,No,No,No,No,Yes
3,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,...,No,No,Steady,No,No,No,No,No,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101762,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,MC,NaN,...,No,No,Steady,No,No,No,No,No,No,Yes
101763,Caucasian,Male,[70-80),NaN,1,1,7,1,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101764,Caucasian,Female,[80-90),NaN,2,3,7,10,MC,Surgery-General,...,No,No,Up,No,No,No,No,No,Ch,Yes


In [33]:
df_diabetes.shape

(99493, 47)

In [34]:
missing_values = df_diabetes.isnull().sum()
missing_values

race                            0
gender                          0
age                             0
weight                      96434
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  39712
medical_specialty           48766
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         19
diag_2                        336
diag_3                       1349
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

# 2. Assumption Checks

## **Normality**

In [35]:
stats.shapiro(df_diabetes['time_in_hospital'])

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.8867098093032837, pvalue=0.0)

In [36]:
groups = df_diabetes.groupby(['race', 'A1Cresult'])

for (race_status, A1Cresult_status), group_df in groups:
    _, p_value = stats.shapiro(group_df['time_in_hospital'])

    print(f"Group ({race_status}, {A1Cresult_status}):")
    print(f"P-value from Shapiro-Wilk Test: {p_value}\n")

Group (AfricanAmerican, >7):
P-value from Shapiro-Wilk Test: 1.8906119489850806e-16

Group (AfricanAmerican, >8):
P-value from Shapiro-Wilk Test: 6.7359146155303186e-34

Group (AfricanAmerican, None):
P-value from Shapiro-Wilk Test: 0.0

Group (AfricanAmerican, Norm):
P-value from Shapiro-Wilk Test: 2.9696700824519264e-25

Group (Asian, >7):
P-value from Shapiro-Wilk Test: 0.0023467366117984056

Group (Asian, >8):
P-value from Shapiro-Wilk Test: 2.233798113593366e-05

Group (Asian, None):
P-value from Shapiro-Wilk Test: 2.6558121925662052e-21

Group (Asian, Norm):
P-value from Shapiro-Wilk Test: 0.0007482930086553097

Group (Caucasian, >7):
P-value from Shapiro-Wilk Test: 1.7925673399754353e-38

Group (Caucasian, >8):
P-value from Shapiro-Wilk Test: 0.0

Group (Caucasian, None):
P-value from Shapiro-Wilk Test: 0.0

Group (Caucasian, Norm):
P-value from Shapiro-Wilk Test: 1.930148504761003e-41

Group (Hispanic, >7):
P-value from Shapiro-Wilk Test: 2.1721559733123286e-06

Group (Hispanic

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


### **Interpretation**

## **Homoscedasticity (Equal Variances)- Levene's Test**

In [37]:
stats.levene(
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'AfricanAmerican'][df_diabetes['A1Cresult'] == '>7'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'AfricanAmerican'][df_diabetes['A1Cresult'] == '>8'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'AfricanAmerican'][df_diabetes['A1Cresult'] == 'None'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'AfricanAmerican'][df_diabetes['A1Cresult'] == 'Norm'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Asian'][df_diabetes['A1Cresult'] == '>7'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Asian'][df_diabetes['A1Cresult'] == '>8'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Asian'][df_diabetes['A1Cresult'] == 'None'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Asian'][df_diabetes['A1Cresult'] == 'Norm'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Caucasian'][df_diabetes['A1Cresult'] == '>7'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Caucasian'][df_diabetes['A1Cresult'] == '>8'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Caucasian'][df_diabetes['A1Cresult'] == 'None'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Caucasian'][df_diabetes['A1Cresult'] == 'Norm'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Hispanic'][df_diabetes['A1Cresult'] == '>7'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Hispanic'][df_diabetes['A1Cresult'] == '>8'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Hispanic'][df_diabetes['A1Cresult'] == 'None'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Hispanic'][df_diabetes['A1Cresult'] == 'Norm'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Other'][df_diabetes['A1Cresult'] == '>7'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Other'][df_diabetes['A1Cresult'] == '>8'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Other'][df_diabetes['A1Cresult'] == 'None'],
    df_diabetes['time_in_hospital'][df_diabetes['race'] == 'Other'][df_diabetes['A1Cresult'] == 'Norm'],
)

LeveneResult(statistic=7.450102003925801, pvalue=9.580811219790571e-21)

### **Interpretation**

# Conduct the ANOVA

In [38]:
model = ols('time_in_hospital ~ C(race) * (A1Cresult)', data=df_diabetes).fit()

anova_table = sm.stats.anova_lm(model, typ =2)
print(anova_table)

                          sum_sq       df           F        PR(>F)
C(race)               641.735130      4.0   18.075566  7.501203e-15
A1Cresult            3895.231358      3.0  146.287793  1.363867e-94
C(race):A1Cresult     276.823623     12.0    2.599070  1.845652e-03
Residual           882895.150091  99473.0         NaN           NaN


### **Interpretation**

P-value is significantly low, therefore one can preform the Tukey HSD Post-hoc Test

# 4. Post-hoc Test

In [41]:
# Conduct Tukey's HSD test

# For A1Cresults
posthoc_a1cresults = pairwise_tukeyhsd(endog=df_diabetes['time_in_hospital'], groups=df_diabetes['A1Cresult'], alpha=0.05)
print("Tukey HSD test for A1C Results:\n", posthoc_a1cresults)

# For  Race
posthoc_race = pairwise_tukeyhsd(endog=df_diabetes['time_in_hospital'], groups=df_diabetes['race'], alpha=0.05)
print("\nTukey HSD test for Race:\n", posthoc_race)

Tukey HSD test for A1C Results:
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    >7     >8   -0.157 0.0397 -0.3089 -0.0051   True
    >7   None   -0.588    0.0 -0.7162 -0.4598   True
    >7   Norm   0.0238  0.983 -0.1425  0.1902  False
    >8   None   -0.431    0.0 -0.5209 -0.3412   True
    >8   Norm   0.1808 0.0046  0.0418  0.3198   True
  None   Norm   0.6118    0.0  0.4993  0.7243   True
----------------------------------------------------

Tukey HSD test for Race:
       Multiple Comparison of Means - Tukey HSD, FWER=0.05       
     group1       group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------------
AfricanAmerican     Asian  -0.5125 0.0002 -0.8396 -0.1855   True
AfricanAmerican Caucasian  -0.1221    0.0 -0.1879 -0.0564   True
AfricanAmerican  Hispanic   -0.448    0.0 -0.6378 -0.2582   True
AfricanAmerican     

### **Interpretation**